# Assignment 1: Transformations


TEAM-ID:   22
TEAM-NAME: grip-force  
YOUR-ID: 
YOUR-NAME:   

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [2]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################
# Replace "pass" statement with your code

#Reading Images
img_color = o3d.io.read_image("color.jpg")
img_depth = o3d.io.read_image("depth.png")

#Creating RGBD
#rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
#    img_color, img_depth)

rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(
        img_color, img_depth)

#create point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))

# Flipping the pointcloud, or it will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

#size of cube
size_cube = 0.5
# pick_points(pcd)
[gx, gy, gz] = [-0.28, -0.85+size_cube/2, -3.7] #point on the 'ground'


# create co-ordinate frame
crd_size = 0.55 #size of coordinate frame
A = o3d.geometry.TriangleMesh.create_coordinate_frame(size = crd_size, origin = [-gx,-gy,-gz])

# create scene.pcd[world frm + pntcld] 
o3d.io.write_point_cloud("scene.pcd", pcd + A.sample_points_uniformly(number_of_points= 500))

def one_one(file_name): #Visualizes scene.pcd
    PntCld = o3d.io.read_point_cloud(file_name)
#     pick_points(PntCld)
    o3d.visualization.draw_geometries([PntCld])
#     vis = o3d.visualization.Visualizer()
#     vis.create_window()
#     vis.add_geometry(pcd)
#     vis.run()
#     vis.destroy_window()


# some points on the floor:
# [Open3D INFO] Picked point #189171 (-0.28, -0.93, -3.3) to add in queue.
# [Open3D INFO] Picked point #193355 (-0.076, -0.95, -3.2) to add in queue.
# [Open3D INFO] Picked point #200426 (-0.28, -0.96, -3) to add in queue.
# [Open3D INFO] Picked point #211726 (-0.28, -0.97, -2.7) to add in queue.
# [Open3D INFO] Picked point #209984 (-0.055, -0.96, -2.7) to add in queue.
# [Open3D INFO] Picked point #218866 (-0.051, -0.97, -2.6) to add in queue.

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [3]:
#uncomment the following and add input parameters if any
one_one("scene.pcd")

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [4]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
# Replace "pass" statement with your code
import time

# pick points function
def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) Afther picking points, press q for close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()



cube = o3d.geometry.TriangleMesh.create_box(width=size_cube, height=size_cube, depth=size_cube)
cube.paint_uniform_color([0.9, 0.9, 0.1])

# cube.compute_vertex_normals()
# create co-ordinate frame
B = o3d.geometry.TriangleMesh.create_coordinate_frame(size = crd_size, origin = [-size_cube/2, -size_cube/2, -size_cube/2]) 
B_cube = B.sample_points_uniformly(number_of_points=1000) + cube.sample_points_uniformly(number_of_points=5000)
# translate cube
B_cube.translate((gx,gy,gz), relative = False)

o3d.io.write_point_cloud("cube.pcd", B_cube)
scene_pcd = o3d.io.read_point_cloud("scene.pcd")
cube_pcd = o3d.io.read_point_cloud("cube.pcd")


# pick_points(scene_pcd)
# one_one("cube.pcd")

#visualization of cube on the ground
# o3d.visualization.draw_geometries([cube_pcd, scene_pcd])
pass


def two_one(E, bgrnd, obj): #non-blocking visualization; animation of cube
#     E = [z,y,x] Euler angles. 'bgnrnd' is background onject(static). obj is object tht has to be rotated
#     copy of input pntclds to preserve them
    obj_temp = copy.deepcopy(obj)
    bgrnd_temp = copy.deepcopy(bgrnd)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(bgrnd_temp)
    vis.add_geometry(obj_temp)
    save_image = False
    wait_time =  0.5 
    
    parameter_indx = 1 #index of parameter
    angle_incrmnt = np.pi/36 #the angle by which we increment the rotation in each 'frame of animation'
    
    for Param in E:
        increments = int(Param/5) #no. of increments
        for angle in range(1,increments+1): #loop runs for increments times,calculates rotation mat for each step
            
            if(parameter_indx == 1):#zrot
                R = obj_temp.get_rotation_matrix_from_zyx( (angle_incrmnt*angle, 0, 0) )
            elif(parameter_indx == 2):#yrot
                R = obj_temp.get_rotation_matrix_from_zyx( (np.deg2rad(E[0]),angle_incrmnt*angle,0) )
            elif(parameter_indx == 3):#xrot
                R = obj_temp.get_rotation_matrix_from_zyx( ( np.deg2rad(E[0]), np.deg2rad(E[1]), angle_incrmnt*angle ) )
            
            obj_temp.rotate(R, center=(gx,gy,gz)) #rotate
            vis.update_geometry(obj_temp) #update geometry
            vis.poll_events()
            vis.update_renderer()
            time.sleep(wait_time)            
            obj_temp.rotate(np.transpose(R), center=(gx,gy,gz)) #undo rotation
            
        parameter_indx += 1
        
    R = obj_temp.get_rotation_matrix_from_zyx( ( np.deg2rad(E[0]), np.deg2rad(E[1]), np.deg2rad(E[2]) ) ) #final position
    obj_temp.rotate(R, center=(gx,gy,gz)) #rotate
    time.sleep(3) #observation time
    vis.destroy_window()
    
    
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [5]:
#uncomment the following and add input parameters if any
two_one([30,90,45],scene_pcd,cube_pcd)

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [6]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [7]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code

def EulerAngle_from_RotationMatrix(M):  
     
    if M[2][0] == -1:
        alpha = 0
        beta = np.pi / 2
        gamma = np.arctan2(M[0][1],M[1][1])
        
    elif M[2][0] == 1:
        alpha = 0
        beta = - np.pi / 2
        gamma = - np.arctan2(M[0][1],M[1][1])           
        
    else:
        beta = np.arctan2(-M[2][0],((M[0][0]**2 + M[1][0]**2)**(0.5))) #in radians
        alpha = np.arctan2(M[1][0]/np.cos(beta),M[0][0]/np.cos(beta))
        gamma = np.arctan2(M[2][1]/np.cos(beta),M[2][2]/np.cos(beta))
       
    return [alpha,beta,gamma]

def RotationMatrix_from_EulerAngles(alpha,beta,gama):
     
    [sa,sb,sc] = np.sin([alpha,beta,gama])
    [ca,cb,cc] = np.cos([alpha,beta,gama])
    
    M = np.array([
    [ca*cb, ca*sb*sc-sa*cc, ca*sb*cc+sa*sc],
    [sa*cb, sa*sb*sc+ca*cc, sa*sb*cc-ca*sc],
    [-sb  , cb*sc         , cb*cc         ]])
    
    return M

M = R.from_matrix(M_given)
[alpha,beta,gama] = EulerAngle_from_RotationMatrix(M_given)
my_array_case1 = np.array([alpha,beta,gama])
M_recovered = RotationMatrix_from_EulerAngles(alpha,beta,gama)

print("Case 1:")
print("Angles as per code from scratch (in radians) [α,β,γ] : ", EulerAngle_from_RotationMatrix(M_given))
print("Angles from scipy function (in radians) [α,β,γ] : ",M.as_euler('zyx'))
print("M_given : ")
print(M_given)
print("M_recovered : ")
print(RotationMatrix_from_EulerAngles(alpha,beta,gama))

N = R.from_matrix(N_given)
[alpha,beta,gamma]=EulerAngle_from_RotationMatrix(N_given)
my_array_case2 = np.array([alpha,beta,gama])
N_recovered = RotationMatrix_from_EulerAngles(alpha,beta,gamma)

print("\n")
print("Case 2:")
print("Angles as per code from scratch (in radians) [α,β,γ] : ", EulerAngle_from_RotationMatrix(N_given))
print("Angles from scipy function (in radians) [α,β,γ] : ",N.as_euler('zyx'))
print("N_given : ")
print(N_given)
print("N_recovered : ")
print(RotationMatrix_from_EulerAngles(alpha,beta,gama))

print("\n")
print("Case 3:")
print("Quaternion of the Rotation matrix : ",M.as_quat())
print("Rotation matrix from the Quaternion \n", R.from_quat(M.as_quat()).as_matrix())

print("\n")
print("Quaternion of the Rotation matrix : ",N.as_quat())
print("Rotation matrix from the Quaternion \n", R.from_quat(N.as_quat()).as_matrix())
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Case 1:
Angles as per code from scratch (in radians) [α,β,γ] :  [0.6981317003487097, 1.2217304765297095, 0.5235987753730347]
Angles from scipy function (in radians) [α,β,γ] :  [ 0.64409997  1.2369806  -0.44174663]
M_given : 
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
M_recovered : 
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]


Case 2:
Angles as per code from scratch (in radians) [α,β,γ] :  [0, 1.5707963267948966, -0.17453292552956243]
Angles from scipy function (in radians) [α,β,γ] :  [ 1.57079633  1.3962634  -1.57079633]
N_given : 
[[ 0.         -0.17364818  0.98480775]
 [ 0.          0.98480775  0.17364818]
 [-1.          0.          0.        ]]
N_recovered : 
[[ 6.12323400e-17  5.00000000e-01  8.66025404e-01]
 [ 0.00000000e+00  8.66025404e-01 -5.00000000e-01]
 [-1.00000000e+00  3.06161700e-17  5.30287619e-17]]


Case 3:
Quaternion of t

## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans:
        - No, we did not use np.arctan() but used np.arctan2() instead.
        - We can get infinite answers in arctan() and we don't know the sign to determine the quadrant, but when we calculate using arctan2() it determines the quardrant in which it is present

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [8]:
# Uncomment and replace my_array_case1 with your array. 
print("My Euler angles for case 1 are" + str(my_array_case1))

My Euler angles for case 1 are[0.6981317  1.22173048 0.52359878]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [9]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans: 
        - No problem in extraction, because the angles have a one to one representation with the matrix and matrix could be easily recovered.

### Repeat the above for Case 2.

In [10]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are" + str(my_array_case2))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[0.         1.57079633 0.52359878]
For case 2, it is True I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: Yes there is problem in representation, Here $\beta$ is 90°, -90° hence we are unable to calcuate $\alpha ,\gamma$ uniquely and we have to assume alpha to be 0° to get value of $\gamma$ from arctan2(). Hence we have problem with Euler angle representation.
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: 
        - We can't recover the exact matrix when we have a gimbal lock or/ and $\beta$ is 90°, -90° because of the loss of one degree of freedom we have infinite solutions for $\alpha ,\gamma$.
        - Gimbal lock is when we fix or align one axis, we lose one degree of freedom and hence no gimbal available to accommodate rotation about one axis. There loss of one degree of freedom in a three-dimensional leads to "locking" the system into rotation in a degenerate two-dimensional space. 

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: Yes we have used 'zyx' for r.as_euler()
            - Extrinsic rotation uses an external world frame fixed axis to rotate an object. (Fixed Frame)
            - Instrisic rotation uses the axis and object frame for rotation. (Moving Frame)
        
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: No warning was shows but they should show a warning because there occurs a gimbal lock condition in N case.
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: Quaternions have no problems of gimbal lock, as it can be scene as having four gimbals. It is better to use quaternion instead of the Euler angles as 3 degrees of freedom are always ensured in the Quaternion from and no degeneracy will occur.


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \textbf{Ans: Your set of equations here} $$
$$ cos\alpha sin\beta sin\gamma - sin\alpha cos\gamma = 1/\sqrt3 $$ 
$$ sin\alpha sin\beta sin\gamma + cos\alpha cos\gamma = 1/\sqrt3 $$ 
$$ cos\beta sin\gamma = 1/\sqrt3 $$ 

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: $a_1, b_1, c_1$ = [ -0.3978 -93.7569   0.7137] 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = your answer here.*

In [11]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [12]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code
def eqn_func(x):
    return [ np.cos(x[0])*np.sin(x[1])*np.sin(x[2]) - np.sin(x[0])*np.cos(x[2]) - 1/np.sqrt(3),
        np.sin(x[0])*np.sin(x[1])*np.sin(x[2]) + np.cos(x[0])*np.cos(x[2]) - 1/np.sqrt(3),
        np.cos(x[1])*np.sin(x[2]) - 1/np.sqrt(3)]



a1,b1,c1 = fsolve(eqn_func, [0,0,0])

visualised_ans = np.array([np.pi/4, 0 , np.pi/6])   #[-π/4, 0, π/2-cos⁻¹(1/√3)]
error = 5 * np.pi/180
a2,b2,c2 = fsolve(eqn_func, [-np.pi/4 - error , 0 + error , np.pi/6 - error])
a2, b2, c2 = fsolve(eqn_func, [-0.78-error, 0+error, 0.62-error])

print("Check if found vector is close by np.isclose() for [a1,b1,c1] : ",np.isclose(eqn_func([a1, b1, c1]), [0.0, 0.0, 0.0]))
print("Check if found vector is close by np.isclose() for [a2,b2,c2] : ",np.isclose(eqn_func([a2, b2, c2]), [0.0, 0.0, 0.0]))
print("[a1,b1,c1] : ",np.around([a1,b1,c1], decimals = 3))
print("[a2,b2,c2] : ",np.around([a2,b2,c2], decimals = 3))


##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Check if found vector is close by np.isclose() for [a1,b1,c1] :  [ True  True  True]
Check if found vector is close by np.isclose() for [a2,b2,c2] :  [ True  True  True]
[a1,b1,c1] :  [ -0.398 -93.757   0.714]
[a2,b2,c2] :  [-0.801 -0.023  0.616]


In [13]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = RotationMatrix_from_EulerAngles(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = RotationMatrix_from_EulerAngles(a2,b2,c2) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [14]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [15]:

##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code

def two_four(Animate_array):
    #define geometries
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
    mesh_sphere.compute_vertex_normals()
    mesh_sphere.paint_uniform_color([0.7, 0.8, 0.3])
    
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2)
    
    mesh_sphere.translate((np.sqrt(4/3),np.sqrt(4/3),np.sqrt(4/3)))
    
    #create copy
    frame_temp = copy.deepcopy(mesh_frame)
    sphere_temp = copy.deepcopy(mesh_sphere)
    
    #open & initialize visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(sphere_temp)
    vis.add_geometry(frame_temp)

    
    #Animation part
#     Animate_array = [[-45,55,20]]

    for i in Animate_array:
        vis.update_geometry(frame_temp) #update geometry
        vis.poll_events()
        vis.update_renderer()
        Animation(i, sphere_temp, frame_temp, vis)  

    
    #destroy visualizer
    vis.destroy_window()

    
    

def Animation(E, bgrnd_temp, obj_temp, visual_screen): #non-blocking visualization; animation of cube
#     E = [z,y,x] Euler angles. 'bgnrnd' is background onject(static). obj is object tht has to be rotated
#     copy of input pntclds to preserve them


    save_image = False
    wait_time =  0.001 
    
    vis = visual_screen
    time.sleep(2) #observation time
    
    parameter_indx = 1 #index of parameter
    angle_incrmnt = np.pi/(36*5) #the angle by which we increment the rotation in each 'frame of animation'
    
    for Param in E:
#         increments = int(Param/5) #no. of increments
        for angle in range(1,abs(Param)+1): #loop runs for increments times,calculates rotation mat for each step
            
            if(parameter_indx == 1):#zrot
                R = obj_temp.get_rotation_matrix_from_zyx( (angle_incrmnt*angle*np.sign(Param), 0, 0) )
            elif(parameter_indx == 2):#yrot
                R = obj_temp.get_rotation_matrix_from_zyx( (np.deg2rad(E[0]),angle_incrmnt*angle*np.sign(Param),0) )
            elif(parameter_indx == 3):#xrot
                R = obj_temp.get_rotation_matrix_from_zyx( ( np.deg2rad(E[0]), np.deg2rad(E[1]), angle_incrmnt*angle*np.sign(Param)))
            
            obj_temp.rotate(R, center=(0,0,0)) #rotate
            vis.update_geometry(obj_temp) #update geometry
            vis.poll_events()
            vis.update_renderer()
            time.sleep(wait_time)            
            obj_temp.rotate(np.transpose(R), center=(0,0,0)) #undo rotation
            
        parameter_indx += 1
        
    R = obj_temp.get_rotation_matrix_from_zyx( ( np.deg2rad(E[0]), np.deg2rad(E[1]), np.deg2rad(E[2]) ) ) #final position
    obj_temp.rotate(R, center=(0,0,0)) #rotate
    time.sleep(2) #observation time
    obj_temp.rotate(np.transpose(R), center=(0,0,0))#unrotate get back to original position
    
    


##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: π/2, -π/2
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: - For $\beta\$ = π/2 the rotation matrix comes out to be:
    
    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & sin(\gamma-\alpha) & cos(\gamma-\alpha) \\0 & cos(\gamma-\alpha) & -sin(\gamma-\alpha) \\
    -1 & 0 & 0\end{array}\right] $$
    
    - For $\beta\$ = -π/2 the rotation matrix comes out to be:
    
    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -sin(\alpha+\gamma) & -cos(\alpha+\gamma) \\0 & cos(\alpha+\gamma) & -sin(\alpha+\gamma) \\
    1 & 0 & 0\end{array}\right] $$
    
    For the above cases, when $\beta$ = π/2 or -π/2, the equation loses one degree of freedom and only depends on $\gamma-\alpha$ or $\gamma+\alpha$ rather than $\alpha,\beta,\gamma$ individually and a gimbal lock is achieved.
    
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [16]:
#uncomment the following and add input parameters if any
two_four([[-45, 0, 35], [-35,90,360], [45,90,-360], [-35,-90,360], [45,-90,-360],[10,90,-360],
          [20,90,-360], [80,90,-360]])
pass